In [1]:
import spacepy.pycdf as pycdf
import sys
sys.path.append('c:/users/cschiff/Documents/GitHub/PAD/')

In [2]:
import PAD
import Grapher

In [3]:
debug_filename = 'C:\Yuggoth\BBF\mms1_fpi_brst_l2_des-debug_20160809092044_v3.1.1.cdf'.replace('\\','/')
dist_filename  = 'C:\Yuggoth\BBF\mms1_fpi_brst_l2_des-dist_20160809092044_v3.1.1.cdf'.replace('\\','/')
photo_filename = 'C:\Yuggoth\Photoelectron Model\mms_fpi_brst_l2_des-bgdist_v1.1.0_p0-2.cdf'.replace('\\','/')

In [4]:
cdf_dict       = {'debug' : pycdf.CDF(debug_filename),
                  'dist'  : pycdf.CDF(dist_filename),
                  'photo' : pycdf.CDF(photo_filename)}
obs            = 'mms1'
mode           = 'brst'
species        = 'des'
ver            = 'ver3'
corrections_on = 0

In [5]:
core_data      = PAD.load_particle_data(cdf_dict,obs,mode,species,ver,corrections_on)

PAD.py:306: RuntimeWarning: invalid value encountered in divide
  counts                   = (dist['Dist']/dist['Err'])**2


In [6]:
core_data.keys()

['bfield', 'jN_err', 'jN', 'v_dirs', 'dist_data', 'counts', 'parms']

A quick check on the pitch angles computed in PAD

In [7]:
my_phi  = core_data['parms']['Phi'][7,5]*np.pi/180.0

In [8]:
my_theta = core_data['parms']['Theta'][6]*np.pi/180.0

In [9]:
my_look = [sp.cos(my_phi)*sp.sin(my_theta),sp.sin(my_phi)*sp.sin(my_theta),sp.cos(my_theta)]

In [10]:
my_look

[0.38349272289401076, 0.87673721127747861, 0.29028467725446233]

In [11]:
print core_data['v_dirs'][7,5,6]
print core_data['v_dirs'][7,5,6] + my_look

[-0.38349272 -0.87673721 -0.29028468]
[  1.11022302e-16  -1.11022302e-16   0.00000000e+00]


In [12]:
num_time_steps = len(core_data['dist_data']['Epoch'])

In [14]:
time_interval_str = 'Burst Interval 08_09_2017 09_20_44'
file_path         = 'c:/Users/cschiff/Documents/GitHub/PAD/'
Elow              = 0
Ehigh             = 31
#for time_label 
time_label        = 0
Grapher.create_raw_survey_PAD_plot(mode,time_label,Elow,Ehigh,time_interval_str,file_path,core_data)

In [15]:
dir(PAD)

['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__warningregistry__',
 'bisect_epochs',
 'compute_counts',
 'compute_incoming_particle_directions',
 'compute_limited_PAD',
 'compute_limited_PAD_error',
 'compute_number_flux',
 'compute_number_flux_errors',
 'compute_omni_number_flux',
 'compute_pitch_angles',
 'detect_DMPA_str',
 'fetch_AFG_B_field',
 'fetch_magnetic_field',
 'load_moments_data',
 'load_particle_data',
 'np',
 'num_az',
 'num_polar',
 'sp',
 'spherical_cap_area',
 'subtract_internal_photoelectrons',
 'unpack_dist_cdf']

In [16]:
dir(Grapher)

['Convert',
 'PAD',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 'calculate_flux_LM',
 'cbar_position',
 'cmap',
 'create_flux_survey_spectrum',
 'create_raw_survey_PAD_plot',
 'create_raw_survey_PAD_plot_LM',
 'create_smooth_survey_PAD_plot',
 'dt',
 'find_sign_intervals',
 'make_brst_summary_plot',
 'make_fast_summary_plot',
 'mdates',
 'np',
 'plt',
 'ticker',
 'visualize_FPI_pixels_in_LM']

In [17]:
core_data['dist_data']['Epoch']

array([datetime.datetime(2016, 8, 9, 9, 20, 44, 11994),
       datetime.datetime(2016, 8, 9, 9, 20, 44, 41994),
       datetime.datetime(2016, 8, 9, 9, 20, 44, 71994), ...,
       datetime.datetime(2016, 8, 9, 9, 21, 39, 932274),
       datetime.datetime(2016, 8, 9, 9, 21, 39, 962274),
       datetime.datetime(2016, 8, 9, 9, 21, 39, 992274)], dtype=object)

In [18]:
cdf_dict['debug']['Epoch'][-1]

datetime.datetime(2016, 8, 9, 9, 21, 39, 992274)

In [19]:
print len(cdf_dict['debug']['Epoch']), len(core_data['dist_data']['Epoch'])

1867 1867


In [20]:
def load_moments_data(obs,curr_debug):
    #unpack the CDF
    times           = np.asarray(curr_debug['Epoch'])
    energies        = np.asarray(curr_debug['%s_des_energy_brst' % obs])[0,:]
    sc_pot          = np.asarray(curr_debug['%s_des_scpot_mean_brst' % obs])
    bx              = np.asarray(curr_debug['%s_des_bentPipeB_X_DSC' % obs])
    by              = np.asarray(curr_debug['%s_des_bentPipeB_Y_DSC' % obs])
    bz              = np.asarray(curr_debug['%s_des_bentPipeB_Z_DSC' % obs])
    bnorm           = np.asarray(curr_debug['%s_des_bentPipeB_Norm' % obs])
    par             = np.asarray(curr_debug['%s_des_energyspectr_par_brst' % obs]).T
    anti            = np.asarray(curr_debug['%s_des_energyspectr_anti_brst' % obs]).T
    perp            = np.asarray(curr_debug['%s_des_energyspectr_perp_brst' % obs]).T    
    temp            = np.asarray(curr_debug['%s_des_energyspectr_omni_brst' % obs])
    omni            = np.ma.masked_invalid(np.log10(temp).T)
    temp            = np.asarray(curr_debug['%s_des_pitchangdist_lowen_brst' % obs])
    low             = np.ma.masked_invalid(np.log10(temp).T)
    temp            = np.asarray(curr_debug['%s_des_pitchangdist_miden_brst' % obs])
    mid             = np.ma.masked_invalid(np.log10(temp).T)
    temp            = np.asarray(curr_debug['%s_des_pitchangdist_highen_brst' % obs])
    high            = np.ma.masked_invalid(np.log10(temp).T)
    angles          = np.linspace(0,180,30)
    ratio           = np.ma.masked_invalid(np.divide(par,anti))
    anti            = np.ma.masked_invalid(np.log10(anti))
    par             = np.ma.masked_invalid(np.log10(par))
    perp            = np.ma.masked_invalid(np.log10(perp))
    
    moments_data             = {}
    moments_data['times']    = times   
    moments_data['energies'] = energies
    moments_data['sc_pot']   = sc_pot  
    moments_data['bx']       = bx      
    moments_data['by']       = by      
    moments_data['bz']       = bz      
    moments_data['bnorm']    = bnorm   
    moments_data['par']      = par     
    moments_data['anti']     = anti    
    moments_data['perp']     = perp    
    moments_data['omni']     = omni    
    moments_data['low']      = low     
    moments_data['mid']      = mid     
    moments_data['high']     = high    
    moments_data['angles']   = angles  
    moments_data['ratio']    = ratio   

    return moments_data

In [48]:
def make_brst_summary_plot(obs,moments_data,time_label):
    
    times    = moments_data['times']   
    energies = moments_data['energies']
    sc_pot   = moments_data['sc_pot']  
    bx       = moments_data['bx']      
    by       = moments_data['by']      
    bz       = moments_data['bz']      
    bnorm    = moments_data['bnorm']   
    par      = moments_data['par']     
    anti     = moments_data['anti']    
    perp     = moments_data['perp']    
    omni     = moments_data['omni']    
    low      = moments_data['low']     
    mid      = moments_data['mid']     
    high     = moments_data['high']    
    angles   = moments_data['angles']  
    ratio    = moments_data['ratio']   
    
    BRST_time = dt.datetime.strftime(times[time_label],'%Y-%m-%d %H%M%S.%f')
    
    ###########################################################################
    #create the figure and axes
    fig = plt.figure(figsize=(32,20))
    #fig.autofmt_xdate()
    
    cbar_off = 0.01
    cbar_wth = 0.01
    cmap.jet.set_bad('k',alpha=1.0)    
    cmap.bwr.set_bad('k',alpha=1.0)    
    
    ###########################################################################
    #0th pane - spectrogram
    min_exp = 4
    max_exp = 8
    ax0     = fig.add_subplot(6,2,1)
    
    #plot the data
    spec_data = ax0.pcolormesh(times,energies,omni,cmap=cmap.jet,vmin=max_exp,vmax=min_exp)
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax0.xaxis.set_major_locator(seconds)
    ax0.xaxis.set_major_formatter(time_format)
    ax0.set_xlabel('Time')
    
    #deal with the y-axis
    ax0.set_yscale('log')
    ax0.set_ylim([energies[0],energies[31]])
    ax0.set_ylabel('Energy (eV)')
    
    #create the colorbar
    log_cbar_span = np.array(range(min_exp,max_exp+1))
    cb_ax0        = fig.add_axes(cbar_position(ax0,cbar_off,cbar_wth))
    spec_cbar     = fig.colorbar(spec_data,cax=cb_ax0,ticks=log_cbar_span,format=ticker.FormatStrFormatter('$10^{%d}$'))
    #cbar_span     = ['%1.0e' % val for val in 10.0**log_cbar_span]
    #cb_ax0.set_yticklabels(cbar_span)
    
    #put on a title
    ax0.set_title('%s on %s' % (obs.upper(),BRST_time))
    
    ax0.plot(times,sc_pot,'k-')
    ax0.axvline(times[time_label],color = 'k')
    
    ###########################################################################
    #1st pane - low energy PAD
    min_exp = int(np.floor(np.min(low)))
    max_exp = int(np.ceil(np.max(low)))
    ax1 = fig.add_subplot(6,2,3)
    
    #plot the data
    PAD_low_dat = ax1.pcolormesh(times,angles,low,cmap=cmap.bwr,vmin=min_exp,vmax=max_exp)
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax1.xaxis.set_major_locator(seconds)
    ax1.xaxis.set_major_formatter(time_format)
    ax1.set_xlabel('Time')
    
    #deal with the y-axis
    ax1.set_yscale('linear')
    ax1.set_ylim([0,180])
    ax1.set_ylabel('Low Pitch Ang (deg)')
    
    #create the colorbar
    log_cbar_span = np.array(range(min_exp,max_exp+1))
    cb_ax1        = fig.add_axes(cbar_position(ax1,cbar_off,cbar_wth))
    PAD_low_cbar  = fig.colorbar(PAD_low_dat,cax=cb_ax1,ticks=log_cbar_span,format=ticker.FormatStrFormatter('$10^{%d}$'))
    ax1.axvline(times[time_label],color = 'k')

     
    ###########################################################################
    #2nd pane - mid energy PAD
    min_exp = int(np.floor(np.min(mid)))
    max_exp = int(np.ceil(np.max(mid))) 
    ax2 = fig.add_subplot(6,2,5)
    
    #plot the data
    PAD_mid_dat = ax2.pcolormesh(times,angles,mid,cmap=cmap.bwr)
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax2.xaxis.set_major_locator(seconds)
    ax2.xaxis.set_major_formatter(time_format)
    ax2.set_xlabel('Time')
    
    #deal with the y-axis
    ax2.set_yscale('linear')
    ax2.set_ylim([0,180])
    ax2.set_ylabel('Mid Pitch Ang (deg)')
    
    #create the colorbar
    log_cbar_span = np.array(range(min_exp,max_exp+1))
    cb_ax2        = fig.add_axes(cbar_position(ax2,cbar_off,cbar_wth))
    PAD_mid_cbar  = fig.colorbar(PAD_mid_dat,cax=cb_ax2,ticks=log_cbar_span,format=ticker.FormatStrFormatter('$10^{%d}$'))
    ax2.axvline(times[time_label],color = 'k')
    

    ###########################################################################
    #3rd pane - high energy PAD
    min_exp = int(np.floor(np.min(high)))
    max_exp = int(np.ceil(np.max(high))) 
    ax3 = fig.add_subplot(6,2,7)
    
    #plot the data
    PAD_high_dat = ax3.pcolormesh(times,angles,high,cmap=cmap.bwr)
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax3.xaxis.set_major_locator(seconds)
    ax3.xaxis.set_major_formatter(time_format)
    ax3.set_xlabel('Time')
    
    #deal with the y-axis
    ax3.set_yscale('linear')
    ax3.set_ylim([0,180])
    ax3.set_ylabel('Hi Pitch Ang (deg)')
    
    #create the colorbar
    log_cbar_span = np.array(range(min_exp,max_exp+1))
    cb_ax3        = fig.add_axes(cbar_position(ax3,cbar_off,cbar_wth))
    PAD_high_cbar = fig.colorbar(PAD_high_dat,cax=cb_ax3,ticks=log_cbar_span,format=ticker.FormatStrFormatter('$10^{%d}$'))
    ax3.axvline(times[time_label],color = 'k')

    ###########################################################################
    #4th pane - ratio of PADs
    ax4 = fig.add_subplot(6,2,9)
    
    #plot the data
    asym_data = ax4.pcolormesh(times,energies,ratio,cmap=cmap.bwr,vmin=0.6,vmax=1.4)
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax4.xaxis.set_major_locator(seconds)
    ax4.xaxis.set_major_formatter(time_format)
    ax4.set_xlabel('Time')
   
    #deal with the y-axis
    ax4.set_yscale('log')
    ax4.set_ylim([energies[0],energies[31]])
    ax4.set_ylabel('Energy (eV)')
    
    #create the colorbar
    cb_ax4          = fig.add_axes(cbar_position(ax4,cbar_off,cbar_wth))
    asym_cbar       = fig.colorbar(asym_data,cax=cb_ax4)
    ax4.axvline(times[time_label],color = 'k')    

    ###########################################################################
    #5th pane - mag field
    ax5 = fig.add_subplot(6,2,11)
    
    #plot the data
    ax5.plot(times,bx*bnorm,label='Bx')
    ax5.plot(times,by*bnorm,label='By')
    ax5.plot(times,bz*bnorm,label='Bz')
    ax5.set_xlim([times[0],times[-1]])
    
    #deal with epochs on the x-axis
    time_format = mdates.DateFormatter('%H:%M:%S')
    seconds     = mdates.SecondLocator(range(0,59),interval = 5,tz=None)
    ax5.xaxis.set_major_locator(seconds)
    ax5.xaxis.set_major_formatter(time_format)
    ax5.set_xlabel('Time')
    
    #deal with the y-axis
    ax5.set_yscale('linear')
    ax5.set_ylabel('Magnetic Field (nT)')
    ax5.axvline(times[time_label],color = 'k')
    
    ax5.legend(bbox_to_anchor=[1.12,1.0])

    ###########################################################################
    #6th pane - energy spectra
    ax6     = fig.add_subplot(6,2,2)
    Q_par   = 10**(par[:,time_label])
    Q_perp  = 10**(perp[:,time_label])
    Q_anti  = 10**(anti[:,time_label])
    
    #plot the data
    par_spec_data  = ax6.loglog(energies,Q_par,'ko-',label='Parallel')
    perp_spec_data = ax6.loglog(energies,Q_perp,'gs-',label='Perpendicular')
    anti_spec_data = ax6.loglog(energies,Q_anti,'r^-',label='Anti-Parallel')
    
    #x-axis
    ax6.set_xlabel('Energy (eV)')
    ax6.set_xlim([50.0,30.0e3])
    
    #deal with the y-axis
    ax6.set_ylabel('PSD (cm^-6 s^-3)')
    
    ax6.legend(bbox_to_anchor=[1.12,1.0])
       
    ###########################################################################
    #7th pane - perpendicular energy spectrum
    #ax7     = fig.add_subplot(6,2,4)
    
    
    ###########################################################################
    #8th pane - anti-parallel energy spectrum
    #ax8     = fig.add_subplot(6,2,6)
    
    
    ###########################################################################
    #9th pane - Differnential Counter-Streaming energy spectrum
    ax9     = fig.add_subplot(6,2,8)
    
    #plot the data
    ax9.semilogx(energies,Q_par - Q_anti,'b.-')
    
    #x-axis
    ax9.set_xlabel('Energy (eV)')
    ax9.set_xlim([50.0,30.0e3])    
    
    #deal with the y-axis
    ax9.set_ylabel('Differential Counter-Streaming \n PSD (cm^-6 s^-3)')
    ax9.axhline(0,color='k')
    ax9.grid('on')
    
    ###########################################################################
    #10th pane - Relative Counter-Streaming energy spectrum
    ax10     = fig.add_subplot(6,2,10)
    
    #plot the data
    ax10.semilogx(energies,np.divide(Q_par,Q_anti),'b.-')
    
    #x-axis
    ax10.set_xlabel('Energy (eV)')
    ax10.set_xlim([50.0,30.0e3])
    
    #deal with the y-axis
    ax10.set_ylabel('Relative Counter-Streaming\n PSD (Par/Anti)')
    ax10.set_ylim([0.2,1.8])
    ax10.axhline(1,color='k')
    ax10.grid('on')    
    
    
    ###########################################################################
    #save the file
    
    #fig.tight_layout()
    filename = 'x:/fpishare/Conrad/PAD_plots/BRST_summary_%s_%s.png' % (obs,BRST_time)
    fig.savefig(filename)
    fig.clf()

In [22]:
#####################################################################################   
def cbar_position(current_ax,offset,cbar_width):
    #get the tuple describing the lower x,y corner position
    #and the width and height
    x_pos, y_pos, width, height = current_ax.get_position().bounds
    return [x_pos + width + offset, y_pos, cbar_width, height]  

In [23]:
moments_data = load_moments_data(obs,cdf_dict['debug'])

C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: divide by zero encountered in log10
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in log10
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: divide by zero encountered in log10
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: divide by zero encountered in log10
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: divide by zero encountered in divide
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: RuntimeWarning: invalid value encountered in divide
C:\Users\cschiff\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: RuntimeWarning: divide

In [ ]:
for i in range(0,len(moments_data['times'])):
    print i
    make_brst_summary_plot(obs,moments_data,i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157

In [43]:
A = moments_data['par'][:,500]
B = moments_data['anti'][:,500]
E = moments_data['energies']

In [35]:
QA = 10.0**A
QB = 10.0**B

In [44]:
for j in range(32):
    print E[j], A[j], QA[j], B[j], QB[j]

12.41 7.7412 5.51058e+07 7.4795 3.01651e+07
15.91 7.7838 6.07861e+07 7.60508 4.02789e+07
20.4 7.69842 4.99369e+07 7.58519 3.84756e+07
26.16 6.85469 7.15636e+06 6.44398 2.7796e+06
33.54 6.03713 1.08926e+06 5.58308 382899.0
43.01 5.3122 205211.0 5.73932 548679.0
55.15 5.29213 195941.0 -- --
70.71 5.16942 147715.0 -- --
90.67 4.87737 75399.3 5.33169 214630.0
116.26 5.10392 127034.0 -- --
149.08 5.85226 711644.0 5.61146 408751.0
191.15 5.83764 688085.0 5.68475 483898.0
245.1 5.87608 751764.0 5.75567 569738.0
314.27 5.9259 843146.0 6.30926 2.03826e+06
402.97 6.19221 1.55671e+06 6.30887 2.03642e+06
516.7 6.51665 3.28586e+06 6.60966 4.07064e+06
662.53 6.66856 4.6619e+06 6.74423 5.54918e+06
849.51 6.58286 3.82697e+06 7.11175 1.29344e+07
1089.27 6.98079 9.56734e+06 6.88689 7.70711e+06
1396.69 7.10803 1.28241e+07 6.89595 7.86956e+06
1790.88 7.18332 1.52519e+07 6.7963 6.25608e+06
2296.32 7.17445 1.49435e+07 6.75671 5.71095e+06
2944.42 7.13364 1.36033e+07 6.66607 4.63527e+06
3775.42 7.01402 1.0328